- First draft of interfaces for models and train/test flow.
- Interfaces for Predictor and the Test flow definitely need some changes to make sure the "history" is correct
- The idea is to be able to swap out different Configs and Models for testing different scenarios

## Data Flow

DataSample
- CSI
- Loc
- Vel

DataWindow
- current DataSample
- previous k DataSamples

PCA
- Fit: CSI Matrix
    - Reduce Overhead? Part of PCA?
- Process: CSI Matrix -> ZDL Matrix

CSI Predictor
- Fit: ZDL Matrix + Loc Matrix? + Vel Matrix?
- Process: DataWindow (single) ->  

Error Compressor
- Includes bit allocation?
- Train: Error Matrix (ZDL Matrix)
- Test: Error Matrix (ZDL Matrix) -> CompressedErrors

## Data Processing


In [3]:
import numpy as np
from dataset import Dataset, load_data
from utils import Config

## Configure
# Use this cfg variable whenever we need to access some constant
cfg = Config(
    num_rx_antennas=1,
    num_tx_antennas=64,
    num_subcarriers=80,
    train_test_split=0.8,
    data_root="data/dataset1"
)

train_set, test_set = load_data("folder")

TypeError: __init__() missing 1 required positional argument: 'pca_model_path'

## Offline Training


In [5]:
from reference_impl import ReferencePCA, ReferenceKmeans, NullPredictor
from model import train_or_load

pca = ReferencePCA()
train_or_load(pca, cfg.pca_path, train_set.csi_samples)  # pca.fit() includes reduce overhead
zdl_train = pca.process(train_set.csi_samples)
zdl_train_windows = pca.process(train_set.csi_windows)

## Train Predictor
predictor = NullPredictor()
train_or_load(predictor, cfg.predictor_path, zdl_train)  # TODO maybe want to also pass in location etc
predicted_zdl = predictor.process(zdl_train)

prediction_error = zdl_train - predicted_zdl

## Error Compression
error_compressor = ReferenceKmeans()
train_or_load(error_compressor, cfg.kmeans_path, prediction_error)

# DL and UL each get trained pca, predictor, and error_compressor

NameError: name 'cfg' is not defined

## Testing

In [ ]:
## Downlink Side
zdl_test = pca.process(test_set.csi_samples)
# Bit allocation?
predicted_zdl_test = predictor.process(zdl_test)
prediction_error_test = zdl_test - predicted_zdl
compressed_error_test = error_compressor.process(predicted_zdl_test)

# CHANNEL: Send compressed_error to UL

## UL Side
ul_predicted_error = error_compressor.decode(compressed_error_test)
ul_predicted_zdl = predictor.process(zdl_test)  # TODO ? Shouldn't be test_set Might need to do this iteratively like loop over each sample and load the History 
ul_reconstructed_zdl = ul_predicted_error + ul_predicted_zdl
ul_predicted_csi = pca.decode(ul_reconstructed_zdl)

## TODO Compute Accuracy 
# test_set.csi_samples == ul_reconstructed_zdl

## Visualize?


## Simulation
- "End to End" Simulation
- Show simulation of entire new "test" path made up of portions of paths that were trained on

In [ ]:
from copy import deepcopy


In [ ]:
dataset = Dataset()

# Load in pretrained components
dl_sim: DLSimulator = DLSimple(cfg, pca, predictor, error_compressor)
ul_sim: ULSimulator = ULSimple(cfg, pca, predictor, error_compressor)

evaluator = Evaluator()

for sample in dataset:
    quantized_error = dl_sim.simulate(sample)
    predicted_csi = ul_sim.simulate(quantized_error)
    evaluator.evaluate(sample, predicted_csi)
    
evaluator.report()
evaluator.visualize()